This was my first solution to the Titanic Problem that scored 0.8 on Kaggle. This is not my final submission; in my final submission I've broken the dataframes by titles and applied random forest to each set independently thereby maintaining the ratio of survived adult males and women that isn't correct in this code. However this gives you a fair idea of who the machine thinks should've been saved on the boat.

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
to_test=data.Survived

In [5]:
data.drop('Survived',1,inplace=True)
df=data.append(test)
df.reset_index(inplace=True)
df.drop('index',inplace=True,axis=1)

In [6]:
df['Title'] = df['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())

In [7]:
grouped = df.groupby(['Sex','Pclass','Title'])

In [8]:
grouped.median()

PassengerId   Age  SibSp  Parch       Fare
Sex    Pclass Title                                                   
female 1      Dona               1306.0  39.0    0.0    0.0  108.90000
              Dr                  797.0  49.0    0.0    0.0   25.92920
              Lady                557.0  48.0    1.0    0.0   39.60000
              Miss                513.0  30.0    0.0    0.0  108.65415
              Mlle                676.5  24.0    0.0    0.0   59.40210
              Mme                 370.0  24.0    0.0    0.0   69.30000
              Mrs                 857.0  45.0    1.0    0.0   78.26670
              the Countess        760.0  33.0    0.0    0.0   86.50000
       2      Miss                606.5  20.0    0.0    0.0   20.25000
              Mrs                 547.0  30.5    1.0    0.0   26.00000
              Ms                  444.0  28.0    0.0    0.0   13.00000
       3      Miss                603.5  18.0    0.0    0.0    8.05000
              Mrs                 658.0  31.0    1.0    1.0   15.50000
              Ms                  980.0   NaN    0.0    0.0    7.75000
male   1      Capt                746.0  70.0    1.0    1.0   71.00000
              Col                 859.0  54.5    0.0    0.0   32.00000
              Don                  31.0  40.0    0.0    0.0   27.72080
              Dr                  661.0  47.0    1.0    0.0   81.85830
              Jonkheer            823.0  38.0    0.0    0.0    0.00000
              Major               493.5  48.5    0.0    0.0   28.52500
              Master              803.0   6.0    1.0    2.0  134.50000
              Mr                  634.0  41.5    0.0    0.0   47.10000
              Sir                 600.0  49.0    1.0    0.0   56.92920
       2      Dr                  358.5  38.5    0.0    0.0   12.25000
              Master              550.0   2.0    1.0    1.0   26.00000
              Mr                  723.5  30.0    0.0    0.0   13.00000
              Rev                 738.0  41.5    0.0    0.0   13.00000
       3      Master              789.0   6.0    3.0    1.0   22.35830
              Mr                  640.5  26.0    0.0    0.0    7.89580

In [9]:
df["Age"] = df.groupby(['Sex','Pclass','Title'])['Age'].transform(lambda x: x.fillna(x.median()))

In [10]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [11]:
df["Fare"] = df.groupby(['Sex','Pclass','Title'])['Fare'].transform(lambda x: x.fillna(x.median()))

In [12]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [13]:
df['Sex'] = df['Sex'].map({'male':1,'female':0})
df['FamilySize'] = df['Parch'] + df['SibSp'] + 1
df['Alone'] = df['FamilySize'].map(lambda s : 1 if s == 1 else 0)
df['Couple'] = df['FamilySize'].map(lambda s : 1 if s==2 else 0)
df['Family'] = df['FamilySize'].map(lambda s : 1 if 3<=s else 0)

In [14]:
df.Embarked.fillna('S',inplace=True)
df.Cabin.fillna('U',inplace=True)
df['Cabin'] = df['Cabin'].map(lambda c : c[0])

In [15]:
df.drop('Name',axis=1,inplace=True)

In [16]:
class_feature = pd.get_dummies(df['Pclass'],prefix="Pclass")
titles_feature = pd.get_dummies(df['Title'],prefix='Title')
embarked_feature = pd.get_dummies(df['Embarked'],prefix='Embarked')
cabin_feature = pd.get_dummies(df['Cabin'],prefix='Cabin')
df = pd.concat([df,cabin_feature],axis=1)
df = pd.concat([df,class_feature],axis=1)
df = pd.concat([df,titles_feature],axis=1)
df = pd.concat([df,embarked_feature],axis=1)
df.drop('Ticket',inplace=True,axis=1)
df.drop('Pclass',inplace=True,axis=1)
df.drop('Title',inplace=True,axis=1)
df.drop('Cabin',inplace=True,axis=1)

In [17]:
df.drop('Embarked',inplace=True,axis=1)

In [18]:
df.head()

,PassengerId,Sex,Age,SibSp,Parch,Fare,FamilySize,Alone,Couple,Family,...,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Embarked_C,Embarked_Q,Embarked_S
0,1,1,22.0,1,0,7.2500,2,0,1,0,...,0,1,0,0,0,0,0,0,0,1
1,2,0,38.0,1,0,71.2833,2,0,1,0,...,0,0,1,0,0,0,0,1,0,0
2,3,0,26.0,0,0,7.9250,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,0,35.0,1,0,53.1000,2,0,1,0,...,0,0,1,0,0,0,0,0,0,1
4,5,1,35.0,0,0,8.0500,1,1,0,0,...,0,1,0,0,0,0,0,0,0,1


In [19]:
data = df.ix[0:890]
test = df.ix[891:1308]

In [20]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier(n_estimators=200)
clf = clf.fit(data, to_test)

In [21]:
features = pd.DataFrame()
features['feature'] = data.columns
features['importance'] = clf.feature_importances_

In [22]:
features.sort(['importance'],ascending=False)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,feature,importance
0,PassengerId,0.144557
5,Fare,0.133630
2,Age,0.130552
34,Title_Mr,0.112156
1,Sex,0.110393
21,Pclass_3,0.044701
31,Title_Miss,0.039361
35,Title_Mrs,0.037674
18,Cabin_U,0.028855
3,SibSp,0.024012


In [23]:
model = SelectFromModel(clf, prefit=True)
training = model.transform(data)
training.shape

(891, 10)

In [24]:
test

,PassengerId,Sex,Age,SibSp,Parch,Fare,FamilySize,Alone,Couple,Family,...,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Title_the Countess,Embarked_C,Embarked_Q,Embarked_S
891,892,1,34.5,0,0,7.8292,1,1,0,0,...,0,1,0,0,0,0,0,0,1,0
892,893,0,47.0,1,0,7.0000,2,0,1,0,...,0,0,1,0,0,0,0,0,0,1
893,894,1,62.0,0,0,9.6875,1,1,0,0,...,0,1,0,0,0,0,0,0,1,0
894,895,1,27.0,0,0,8.6625,1,1,0,0,...,0,1,0,0,0,0,0,0,0,1
895,896,0,22.0,1,1,12.2875,3,0,0,1,...,0,0,1,0,0,0,0,0,0,1
896,897,1,14.0,0,0,9.2250,1,1,0,0,...,0,1,0,0,0,0,0,0,0,1
897,898,0,30.0,0,0,7.6292,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
898,899,1,26.0,1,1,29.0000,3,0,0,1,...,0,1,0,0,0,0,0,0,0,1
899,900,0,18.0,0,0,7.2292,1,1,0,0,...,0,0,1,0,0,0,0,1,0,0
900,901,1,21.0,2,0,24.1500,3,0,0,1,...,0,1,0,0,0,0,0,0,0,1


In [25]:
test = test.fillna(method='ffill')

In [26]:
testing = model.transform(test)
testing.shape

(418, 10)

In [27]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.cross_validation import cross_val_score
forest = RandomForestClassifier(max_features='sqrt')

parameter_grid = {'max_depth':[5],'n_estimators': [220],'criterion': ['gini','entropy']}

cross_validation = StratifiedKFold(to_test, n_folds=5)

grid_search = GridSearchCV(forest,
                           param_grid=parameter_grid,
                           cv=cross_validation)

grid_search.fit(training, to_test)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Best score: 0.8125701459034792
Best parameters: {'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 220}


In [28]:
pipeline = grid_search
output = pipeline.predict(testing).astype(int)
df_output = pd.DataFrame()
df_output['PassengerId'] = test['PassengerId']
df_output['Survived'] = output
df_output[['PassengerId','Survived']].to_csv('output1.csv',index=False)